<img src="nebrija.jpg" style="width: 300px;" />


### MASTER DATA SCIENCE & BUSINESS ANALYTICS
**UNIVERSIDAD DE NEBRIJA**
# CASO PRÁCTICO FINAL

## INTRODUCCIÓN
Trabajamos como científicos de datos para una empresa de retail que, debido al cambio en los hábitos de consumo de los clientes, está potenciando ampliamente el servicio de venta online. La empresa quiere realizar un modelo de aprendizaje automático para clasificar a los clientes en función de la probabilidad de generar ingresos al comprar en la web.

El objetivo es realizar una serie de acciones específicas para los clientes que es más probable que hagan compras en la web.

Para ello, la empresa ha ido recopilando datos con su herramienta de Google Analytics. Cuenta con datos de sesiones, cada una de ellas de un cliente distinto en un periodo de un año.

### INFORMACIÓN DE LOS DATOS

El conjunto de datos consta de diez atributos numéricos y ocho categóricos.

El atributo `revenue` puede ser usado como la etiqueta de la clase.

Las variables `Administrative`, `administrative duration`, `informational`, `informational duration`, `product related` y `product related duration` representan el número de diferentes tipos de páginas visitadas por el visitante en esa sesión y el tiempo total dedicado a cada una de estas categorías de páginas. Los valores de estas características se derivan de la información del URL de las páginas visitadas por el usuario y se actualizan en tiempo real cuando el usuario realiza una acción, por ejemplo, pasar de una página a otra.

Las características `bounce rate`, `exit rate` y `page value` representan las métricas medidas por Google Analytics para cada página del sitio de comercio electrónico.

- El valor de la característica `bounce rate` de una página web se refiere al porcentaje de visitantes que entran en el sitio desde esa página y luego salen (rebote) sin activar ninguna otra solicitud al servidor de análisis durante esa sesión.
- El valor de la característica `exit rate` para una página web específica se calcula como para todas las visitas a la página, el porcentaje que fueron las últimas en la sesión.
- La función `Page Value` representa el valor medio de una página web que un usuario ha visitado antes de completar una transacción de comercio electrónico.

La característica de `special day` indica la cercanía de la hora de visita del sitio a un día especial específico (por ejemplo, el Día de la Madre, San Valentín) en el que es más probable que las sesiones finalicen con una transacción. El valor de este atributo se determina teniendo en cuenta la dinámica del comercio electrónico, como la duración entre la fecha del pedido y la fecha de entrega. Por ejemplo, para San Valentín, este valor toma un valor distinto de 0 entre el 2 y el 12 de febrero, 0 antes y después de esta fecha a menos que esté cerca de otro día especial, y su valor máximo de 1 el 8 de febrero.

El conjunto de datos también incluye el sistema operativo, el navegador, la región, el tipo de tráfico, el tipo de visitante como visitante que regresa o como nuevo visitante, un valor booleano que indica si la fecha de la visita es de fin de semana, y el mes del año.

### TRABAJO A REALIZAR
Las siguientes son las tareas a realizar. 

1. Realización de un análisis de las variables del dataset de Google Analytics como pueden ser histogramas, boxplots, etc. Cualquier otro análisis es bienvenido, siempre explicándolo y con un sentido de negocio.
2. Tratamiento de los valores faltantes, outliers, etc., en caso de que los hubiese. Si hay valores con missings, habrá que eliminarlos con el método de Pandas llamado `Dropna().a`.
3. Tratamiento de categóricas, pasándolas a numéricas por medio de _dummies_, mapeándolas o utilizando un `label encoder`. Hay que justificar las operaciones que se realizan.
4. Si existe alguna variable que se necesite borrar, habrá que borrara y justificarlo.
5. Estandarizar los datos.
6. Dividir los datos en train y en test. Con los datos de train se pretende ajustar modelos con `CrossValidation` y `GridSearch`.
    - Utilizar un modelo lineal. Entre los modelos lineales están las regresiones logísticas, las regresiones lineales, etc.
    - Utilizar un modelo de redes neuronales.
    - Utilizar cualquier otro modelo de clasificación.
7. Optimizar algún parámetro de cada modelo utilizando `CrossValidation` y `GridSearch`, o de la forma que se estime oportuna, siempre justificándolo.
8. Elegir el mejor modelo de los tres según la métrica ROC en `CrossValidation`. Predecir Test y obtener una métrica estimada.
9. Umbralizar las probabilidades utilizando el umbral que maximice el área bajo la curva ROC.
10. El entregable final será un Jupyter Notebook en el que se realicen todos los análisis y los modelos.

**Anotación**
Lo que se pide es un modelo _end-to-end_ como los vistos en prácticas en el módulo. En este caso, hay que aplicar tres modelos y escoger el mejor, pero la parte previa es común a los tres.

Se tendrán en cuenta la estructura del código, las interpretaciones y las justificaciones.

El esquema que se pone en las tareas es un esquema general. Si se necesita hacer alguna tarea, se puede llevar a cabo. También es posible aplicar cualquier otro algoritmo visto en el módulo, siempre justificando y haciendo las cosas con un sentido.

# SOLUCIÓN PROPUESTA

## Lectura del Juego de Datos
Comenzamos el trabajo con las tareas más esenciales de cualquier juego de dato, la lectura y limpieza del mismo para poder trabajar el análisis EDA con la certeza que no estamos asumiendo tendencias de los datos resultados de artefactos de las impureza de los mismos (Matsui E. y Peng R., 2017). 

In [1]:
import pandas as pd
raw_data = pd.read_csv("online_shoppers_intention.csv")
raw_data.head()

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,Month,OperatingSystems,Browser,Region,TrafficType,VisitorType,Weekend,Revenue
0,0.0,0.0,0.0,0.0,1.0,0.000000,0.20,0.20,0.0,0.0,Feb,1,1,1,1,Returning_Visitor,False,False
1,0.0,0.0,0.0,0.0,2.0,64.000000,0.00,0.10,0.0,0.0,Feb,2,2,1,2,Returning_Visitor,False,False
2,0.0,-1.0,0.0,-1.0,1.0,-1.000000,0.20,0.20,0.0,0.0,Feb,4,1,9,3,Returning_Visitor,False,False
3,0.0,0.0,0.0,0.0,2.0,2.666667,0.05,0.14,0.0,0.0,Feb,3,2,2,4,Returning_Visitor,False,False
4,0.0,0.0,0.0,0.0,10.0,627.500000,0.02,0.05,0.0,0.0,Feb,3,3,1,4,Returning_Visitor,True,False


In [2]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12330 entries, 0 to 12329
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Administrative           12316 non-null  float64
 1   Administrative_Duration  12316 non-null  float64
 2   Informational            12316 non-null  float64
 3   Informational_Duration   12316 non-null  float64
 4   ProductRelated           12316 non-null  float64
 5   ProductRelated_Duration  12316 non-null  float64
 6   BounceRates              12316 non-null  float64
 7   ExitRates                12316 non-null  float64
 8   PageValues               12330 non-null  float64
 9   SpecialDay               12330 non-null  float64
 10  Month                    12330 non-null  object 
 11  OperatingSystems         12330 non-null  int64  
 12  Browser                  12330 non-null  int64  
 13  Region                   12330 non-null  int64  
 14  TrafficType           

Antes de revisar los tipos de columna y transformaciones necesarias, nos asesoramos que no existan datos faltantes. 

In [3]:
raw_data.isnull().sum()

Administrative             14
Administrative_Duration    14
Informational              14
Informational_Duration     14
ProductRelated             14
ProductRelated_Duration    14
BounceRates                14
ExitRates                  14
PageValues                  0
SpecialDay                  0
Month                       0
OperatingSystems            0
Browser                     0
Region                      0
TrafficType                 0
VisitorType                 0
Weekend                     0
Revenue                     0
dtype: int64

Existen 14 registros faltantes en varias de las columnas. El número es el mismo en todas, por lo que puede que existan 14 filas con datos mal registrados. Para un juego de datos de 12,330 filas es una razón bastante pequeña, y si pudieramos eliminarlos ya, nos ahorraría mucho trabajo de imputación de datos innecesario. Revisemos las 14 filas bajo escrutinio. 

In [4]:
# Código extraído de https://stackoverflow.com/questions/30447083/python-pandas-return-only-those-rows-which-have-missing-values
null_data = raw_data[raw_data.isnull().any(axis=1)]
null_data

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,Month,OperatingSystems,Browser,Region,TrafficType,VisitorType,Weekend,Revenue
1065,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,Mar,2,2,2,1,Returning_Visitor,False,False
1132,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,Mar,1,1,1,2,Returning_Visitor,False,False
1133,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,Mar,2,4,5,1,Returning_Visitor,False,False
1134,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,Mar,2,2,1,2,Returning_Visitor,False,False
1135,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,Mar,3,2,1,1,Returning_Visitor,False,False
1136,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,Mar,2,2,1,2,Returning_Visitor,False,False
1473,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,Mar,2,2,1,1,Returning_Visitor,True,False
1474,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,Mar,1,1,6,1,Returning_Visitor,True,False
1475,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,Mar,2,2,3,1,Returning_Visitor,False,False
1476,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,Mar,1,1,2,3,Returning_Visitor,False,False


Los 14 registros parecen estar relativamente juntos, y pudieramos especular que fue un usuario con algún tipo de software para bloquear cookies o trackers. Dado que solo afecta al 0.11% de los registros, procedemos a depurar el juego de datos. 

In [5]:
raw_data.drop(labels=null_data.index, inplace=True, axis=0)
raw_data.isnull().sum()

Administrative             0
Administrative_Duration    0
Informational              0
Informational_Duration     0
ProductRelated             0
ProductRelated_Duration    0
BounceRates                0
ExitRates                  0
PageValues                 0
SpecialDay                 0
Month                      0
OperatingSystems           0
Browser                    0
Region                     0
TrafficType                0
VisitorType                0
Weekend                    0
Revenue                    0
dtype: int64

Nos interesa ver los rangos preliminares de la información cuantitativa, para tomar una idea qué más necesita cambio en el juego de datos. 

In [6]:
raw_data.describe()

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,OperatingSystems,Browser,Region,TrafficType
count,12316.000000,12316.000000,12316.000000,12316.000000,12316.000000,12316.000000,12316.000000,12316.000000,12316.000000,12316.000000,12316.000000,12316.000000,12316.000000,12316.000000
mean,2.317798,80.906176,0.503979,34.506387,31.763884,1196.037057,0.022152,0.043003,5.895952,0.061497,2.124147,2.357584,3.148019,4.070477
std,3.322754,176.860432,1.270701,140.825479,44.490339,1914.372511,0.048427,0.048527,18.577926,0.199020,0.911566,1.718028,2.402211,4.024598
min,0.000000,-1.000000,0.000000,-1.000000,0.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000
25%,0.000000,0.000000,0.000000,0.000000,7.000000,185.000000,0.000000,0.014286,0.000000,0.000000,2.000000,2.000000,1.000000,2.000000
50%,1.000000,8.000000,0.000000,0.000000,18.000000,599.766190,0.003119,0.025124,0.000000,0.000000,2.000000,2.000000,3.000000,2.000000
75%,4.000000,93.500000,0.000000,0.000000,38.000000,1466.479902,0.016684,0.050000,0.000000,0.000000,3.000000,2.000000,4.000000,4.000000
max,27.000000,3398.750000,24.000000,2549.375000,705.000000,63973.522230,0.200000,0.200000,361.763742,1.000000,8.000000,13.000000,9.000000,20.000000


Definitivamente el juego de datos no tiene volumen pero tiene variedad de columnas con diferentes tipos de datos que debemos profundizar un poco antes de seguir. Lo que nos interesa más aquí es el tipo de datos, rango, y que transformación sería la mejor antes de pasar al EDA. 

### Revisión de Tipo de Páginas
El tipo de páginas debe ser una variable importante, ya que tiene su propia columna segregada por _administrativa_, _informativa_ y _producto_. Nos interesa ver si son valores continuos o discretos, si los debemos tratar como valores de estudio o etiquetas de estudio.

In [7]:
web_pages = ["Administrative", 'Administrative_Duration', 'Informational', 'Informational_Duration', 'ProductRelated', 'ProductRelated_Duration']
for pages in web_pages:
    print(u'Page ',pages,' - ', raw_data[pages].nunique(),'\n')

Page  Administrative  -  27 

Page  Administrative_Duration  -  3336 

Page  Informational  -  17 

Page  Informational_Duration  -  1259 

Page  ProductRelated  -  311 

Page  ProductRelated_Duration  -  9552 



En conclusión, hay 27 páginas administrativas, 17 de información, pero muchísimas de producto (311 para ser exactos). No hay mucho más que podamos inferir salvo que las páginas debieran pensarse como variables categóricas ordenadas (ordinales), por lo que preferimos moldearlas (cast) como enteros (Sharma, M., 2020). 

In [8]:
# Código extraído de https://www.linkedin.com/pulse/change-data-type-columns-pandas-mohit-sharma/
raw_data[['Administrative', 'Informational', 'ProductRelated']] = raw_data[['Administrative', 'Informational', 'ProductRelated']].apply(pd.to_numeric, downcast='integer')
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12316 entries, 0 to 12329
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Administrative           12316 non-null  int8   
 1   Administrative_Duration  12316 non-null  float64
 2   Informational            12316 non-null  int8   
 3   Informational_Duration   12316 non-null  float64
 4   ProductRelated           12316 non-null  int16  
 5   ProductRelated_Duration  12316 non-null  float64
 6   BounceRates              12316 non-null  float64
 7   ExitRates                12316 non-null  float64
 8   PageValues               12316 non-null  float64
 9   SpecialDay               12316 non-null  float64
 10  Month                    12316 non-null  object 
 11  OperatingSystems         12316 non-null  int64  
 12  Browser                  12316 non-null  int64  
 13  Region                   12316 non-null  int64  
 14  TrafficType           

### Mapeando Variables Categóricas a Numéricas
Existen dos variables adicionales que debemos mapear de categóricas a numéricas. Una es `Month`, o el mes del año, que es bastante sencilla de interpretar, pero la siguiente es `Weekend`, un indicador de si es o no fin de semana, y `VisitorType`, que nos gustaría verificar antes de aplicar un mapeo. 

In [9]:
raw_data['VisitorType'].unique()

array(['Returning_Visitor', 'New_Visitor', 'Other'], dtype=object)

No tiene mucho sentido el orden en las etiquetas, por lo que podemos establecer uno cualquiera en un mapa antes de proceder con la transformación (Exheen, B., 2017) 

In [10]:
# Código extraído de https://benalexkeen.com/mapping-categorical-data-in-pandas/
raw_data['VisitorType'] = raw_data['VisitorType'].astype("category").cat.codes

Procedemos a verificar los valores de la columna `Weekend`. 

In [16]:
raw_data['Weekend'].unique()

array([False,  True])

Con dos valores del tipo _Boolean_, el mapeo será directo como 0 si es falso y 1 si es verdadero.

In [17]:
raw_data['Weekend'] = raw_data['Weekend'].astype("category").cat.codes
raw_data['Weekend'].unique()

array([0, 1], dtype=int8)

Antes de proceder a crear un mapa de los meses, revisemos la validez de los datos en la columna mes, ya que sin conocer las etiquetas puede ser difícil mapear las mismas. 

In [11]:
raw_data['Month'].unique()

array(['Feb', 'Mar', 'May', 'Oct', 'June', 'Jul', 'Aug', 'Nov', 'Sep',
       'Dec'], dtype=object)

Los valores únicos del atributo mes tienen formatos poco consistentes. Tendremos que crear un diccionario a mano para etiquetarlos como valores numéricos de forma segura (Exhenn, B., 2017). Aquí lo más importante en respecto al código es que `astype` en **Pandas** cambió hace dos años y los parámetros antiguos, sobre todo `ordered=True` ya no funcionan. Lo conveniente es utilizar `CategoricalDType` si el científico de datos le gusta crear mapas a mano (StackOverflow, 2018). En nuestro caso en particular utilizaremos un diccionario sencillo de mapeo. 

In [12]:
# NOTA: astype cambio en Pandas hace poco, utilizar CategoricalDType 
meses_ordenados = {"Month" :
                   {
                       'Jan' : 1,
                       'Feb' : 2,
                       'Mar' : 3,
                       'Apr' : 4,
                       'May' : 5,
                       'June' : 6,
                       'Jul' : 7,
                       'Aug' : 8,
                       'Sep' : 9,
                       'Oct' : 10,
                       'Nov' : 11,
                       'Dec' : 12}}

raw_data = raw_data.replace(meses_ordenados)

In [19]:
raw_data.describe()

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,Month,OperatingSystems,Browser,Region,TrafficType,VisitorType,Weekend
count,12316.000000,12316.000000,12316.000000,12316.000000,12316.000000,12316.000000,12316.000000,12316.000000,12316.000000,12316.000000,12316.000000,12316.000000,12316.000000,12316.000000,12316.000000,12316.000000,12316.000000
mean,2.317798,80.906176,0.503979,34.506387,31.763884,1196.037057,0.022152,0.043003,5.895952,0.061497,7.657113,2.124147,2.357584,3.148019,4.070477,1.718009,0.232624
std,3.322754,176.860432,1.270701,140.825479,44.490339,1914.372511,0.048427,0.048527,18.577926,0.199020,3.391314,0.911566,1.718028,2.402211,4.024598,0.691086,0.422522
min,0.000000,-1.000000,0.000000,-1.000000,0.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,2.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,7.000000,185.000000,0.000000,0.014286,0.000000,0.000000,5.000000,2.000000,2.000000,1.000000,2.000000,2.000000,0.000000
50%,1.000000,8.000000,0.000000,0.000000,18.000000,599.766190,0.003119,0.025124,0.000000,0.000000,7.000000,2.000000,2.000000,3.000000,2.000000,2.000000,0.000000
75%,4.000000,93.500000,0.000000,0.000000,38.000000,1466.479902,0.016684,0.050000,0.000000,0.000000,11.000000,3.000000,2.000000,4.000000,4.000000,2.000000,0.000000
max,27.000000,3398.750000,24.000000,2549.375000,705.000000,63973.522230,0.200000,0.200000,361.763742,1.000000,12.000000,8.000000,13.000000,9.000000,20.000000,2.000000,1.000000


### Estandarización del Juego de Datos
Es probable que para facilitar la tarea de clasificadores y diferentes modelos de aprendizaje automatizado sea conveniente estandarizar ciertos datos. Ahora bien, en nuestro juego de datos muchas columnas son numéricas pero sus valores son etiquetas numéricas de datos categóricos (por ejemplo `Browser` es solo el tipo de explorador que usa el usuario, no una valor contínuo de medición). Solo estaremos estandarizando aquellos datos que tengan sentido, para ser precisos:

- Administrative_Duration
- Informational_Duration
- ProductRelated_Duration
- BounceRates
- ExitRates
- PageValues
- SpecialDay

In [22]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
lista = ['Administrative_Duration', 'Informational_Duration', 
         'ProductRelated_Duration', 'BounceRates', 'ExitRates', 'PageValues', 'SpecialDay']

raw_data[lista] = StandardScaler().fit_transform(raw_data[lista])
raw_data[lista].describe()

## Análisis Explorativo Visual (EDA)